<a href="https://colab.research.google.com/github/silvia-j-escobar/ExternDataScience/blob/main/Copy_of_Table_extraction_from_sample_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Install PyMuPDF (if not already installed)
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 23.8 MB/s eta 0:00:00


In [2]:
# STEP 2: Upload the PDF
from google.colab import files
uploaded = files.upload()


Saving foo.pdf to foo.pdf


In [3]:
# STEP 3: Import PyMuPDF and extract tables
import fitz  # PyMuPDF

In [4]:
def extract_tables_with_bbox(pdf_path):
    doc = fitz.open(pdf_path)
    results = []

    for page_num in range(len(doc)):
        page = doc[page_num]
        tables = page.find_tables()

        for i, table in enumerate(tables.tables):
            table_bbox = table.bbox
            table_data = []
            cell_bboxes = []

            for row_num in range(table.row_count):
                row_data = []
                row_bboxes = []

                for col_num in range(table.col_count):
                    cell_idx = row_num * table.col_count + col_num

                    if cell_idx < len(table.cells) and table.cells[cell_idx] is not None:
                        cell_bbox = table.cells[cell_idx]
                        cell_text = page.get_text("text", clip=cell_bbox).strip()
                        row_data.append(cell_text)
                        row_bboxes.append(cell_bbox)
                    else:
                        row_data.append("")
                        row_bboxes.append(None)

                table_data.append(row_data)
                cell_bboxes.append(row_bboxes)

            results.append({
                'page': page_num,
                'table_index': i,
                'bbox': table_bbox,
                'data': table_data,
                'cell_bboxes': cell_bboxes
            })

    doc.close()
    return results


In [5]:
# STEP 4: Call the function on the uploaded file
pdf_filename = list(uploaded.keys())[0]
results = extract_tables_with_bbox(pdf_filename)


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [6]:
# STEP 5: Print extracted data
for table in results:
    print(f"\n📄 Page {table['page']}, Table {table['table_index']}")
    for row in table['data']:
        print(row)


📄 Page 0, Table 0
['Cycle \nName', '2012_2', '2145_1', '4234_1', '2032_2', '4171_1', 'KI \n(1/km)']
['3.30', '0.68', '0.59', '0.17', '0.07', 'Distance \n(mi)', '1.3']
['11.2', '58.7', '57.8', '173.9', 'Percent Fuel Savings', 'Improved \nSpeed', '5.9%']
['2.4%', '8.5%', '21.7%', '58.1%', 'Decreased \nAccel', '9.5%', '0.1%']
['1.3%', '0.3%', '1.6%', 'Eliminate \nStops', '29.2%', '9.5%', '8.5%']
['2.7%', '2.1%', 'Decreased \nIdle', '17.4%', '2.7%', '3.3%', '1.2%']
['0.5%', '', '', '', '', '', '']
